Задача тематическое моделирование

продолжаем исследование датасета с твитами

Скачиваем датасет (источник): положительные, отрицательные.

рабочие ссылки с твитами
https://disk.yandex.ru/i/v5HM-ENiGXZVpQ
https://disk.yandex.ru/i/koR5eMCToCZS2Q

как альтернатива можно скачать данные из Роспотребнадзора
https://zpp.rospotrebnadzor.ru/Forum/Appeals
для этого берём ноутбук parse_rospotrebnadzor.ipynb
устанавливаем количество скачанных страниц больше не 50-сят хотябы 500 и для анализа берём только вопросы так как ответы есть не всегда

что надо сделать
1. объединить в одну выборку (это только для твитов), для роспотребнадзора сформировать датасет из вопросов
2. провести исследование и выявить тематики о которых говорят в твитах (для твитов), а для роспотребнадзора так же выявить тематики о которых люди пишут проанализировать
3. сделать визуализацию кластеров тематик
4. проинтерпритировать получившиеся тематики


In [1]:
!pip install pymorphy2 pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install -U gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 1.8 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [3]:
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

In [4]:
import pandas as pd
import numpy as np
import re

from nltk.corpus import stopwords
import nltk

import pymorphy2
from gensim.models import *
from gensim import corpora

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
positive = pd.read_csv('./drive/MyDrive/positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)

negative = pd.read_csv('./drive/MyDrive/negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)

df = pd.concat([positive, negative], ignore_index=True)
df.head()

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive


In [7]:
words_regex = re.compile(r'\w+')

def find_words(text, regex = words_regex):
    tokens = regex.findall(text) 
    return [w for w in tokens if w.isalpha() and len(w)>= 4]

morph = pymorphy2.MorphAnalyzer() 
stopwords_list = stopwords.words('russian')

# лемматизация 
def lemmatize(words, lemmer = morph, stopwords = stopwords_list):
    lemmas = [lemmer.parse(w)[0].normal_form for w in words] #список нормальных форм
    return [w for w in lemmas if not w in stopwords and w.isalpha()] #чистим список токенов

#дополнительно почистим токены от мусора
def clear_tokens(tokens):
    string_tokens = ' '.join(tokens)
    ignore_list = [r'[A-za-z]',r'\w+ще\b',r'который',r'\bидио\w+',r'почему',
                   r'\bпид\w+',r'весь',r'значит',r'\bхуй\w+',r'\bхуй\w+',r'\w+xу.\b',
                   r'\bбл\w+',r'\w+xу\w+',r'\w+но\b',r'очень',r'\bпиз\w+',r'это',
                   r'\bгов\w+',r'\bпи.де\w+',r'лох',r'нету',r'ой',r'мочь',r'\w+ть\b',
                   r'\bфиг\w+',r'сука',r'\bпоп\w+',r'свой',r'капец']
    for i in ignore_list:
      string_tokens = re.sub(i, ' ',  string_tokens)

    clear_tokens = string_tokens.split()
    clear_tokens = [w for w in clear_tokens if len(w)>2]
    return clear_tokens

def preprocess(text):
    return (clear_tokens(lemmatize(find_words(text))))

/usr/local/lib/python3.7/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.7/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.7/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.7/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.g

In [8]:
%%time
df['text'] = df['text'].apply(preprocess)
df.head()

CPU times: user 5min 19s, sys: 1.88 s, total: 5min 21s
Wall time: 5min 41s


### Latent Dirichlet Allocation (LDA) 

In [9]:
dictionary = corpora.Dictionary(df['text']) #создаем словарь между токенами и их индексами
dictionary.filter_extremes(no_below = 10, no_above = 0.9, keep_n=None) # игнорируем слова, которые встречаются реже 10 раз или составляют более 0.9 словаря 
dictionary.save('tweet.dict')
len(dictionary.keys())

8184

In [10]:
corpus = [dictionary.doc2bow(text) for text in df['text']] #для каждого списка токенов возвращает кортеж (индекс токена, количество встречаний токена в твите)
corpora.MmCorpus.serialize('tweet.model', corpus) #выполняем итерацию по корпусу


In [14]:
%%time
lda = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=5, chunksize=50, update_every=1, passes=2)

CPU times: user 1min 46s, sys: 474 ms, total: 1min 46s
Wall time: 1min 47s


In [15]:
lda.show_topics(num_topics=5, num_words=20, formatted=True)

[(0,
  '0.031*"жизнь" + 0.030*"время" + 0.023*"мама" + 0.021*"друг" + 0.021*"каждый" + 0.019*"работа" + 0.019*"любимый" + 0.017*"вчера" + 0.017*"прийти" + 0.012*"слово" + 0.011*"папа" + 0.011*"никак" + 0.009*"глаз" + 0.009*"слеза" + 0.009*"зима" + 0.008*"интернет" + 0.008*"нога" + 0.008*"история" + 0.007*"выспаться" + 0.007*"россия"'),
 (1,
  '0.060*"день" + 0.055*"завтра" + 0.032*"школа" + 0.025*"жаль" + 0.021*"урок" + 0.019*"ночь" + 0.019*"последний" + 0.016*"скоро" + 0.016*"первый" + 0.015*"втор" + 0.014*"остаться" + 0.014*"экзамен" + 0.013*"найти" + 0.012*"казаться" + 0.010*"конец" + 0.010*"уйти" + 0.009*"столько" + 0.009*"температура" + 0.008*"таки" + 0.008*"место"'),
 (2,
  '0.019*"час" + 0.017*"голова" + 0.016*"настроение" + 0.015*"новый" + 0.014*"наш" + 0.014*"выходн" + 0.013*"жалко" + 0.012*"снова" + 0.011*"год" + 0.011*"бояться" + 0.011*"вроде" + 0.011*"минута" + 0.011*"рука" + 0.011*"город" + 0.011*"ранний" + 0.010*"маленький" + 0.009*"снег" + 0.009*"парень" + 0.009*"месяц" 

In [16]:
print(lda.log_perplexity(corpus))
print('Персплексия: ', np.exp(lda.log_perplexity(corpus)))

-8.519233069870545
Персплексия:  0.00019958847406332804


In [17]:
visual_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(visual_data)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)
